In [ ]:
import altair as alt
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker
from src.spinorama.contour import compute_contour, compute_contour_smoothed, compute_isoline, reshape
from src.spinorama.graph import graph_freq, graph_contour
from datas.metadata import speakers_info
import numpy as np

df = parse_all_speakers(speakers_info)

In [ ]:
#parse_graphs_speaker('Emotiva Airmotive 6s')

In [ ]:
for k in df.keys():
  print(k)

In [ ]:
neumann = df['Neumann KH 80']['ASR']['default']
for k in neumann.keys():
    print(k)

In [ ]:
spinorama_u = neumann['CEA2034_unmelted']
for m in spinorama_u.keys():
    print(m)
spinorama = neumann['CEA2034']
for m in spinorama.keys():
    print(m)

In [ ]:
onaxis = spinorama.loc[spinorama['Measurements']=='On Axis']
print(onaxis)

In [ ]:
from src.spinorama.analysis import estimates

e0, e3, e6, delta = estimates(onaxis)

print('Estimated -3dB point at %1.fHz'% e3)
print('Estimated -6dB point at %1.fHz'% e6)
print('Estimated +/-%1.1fdB from 80hz to 20kz'%delta)


In [ ]:
from src.spinorama.display import *
dfs = neumann
spinorama = display_spinorama(dfs)
onaxis    = display_onaxis(dfs)
inroom    = display_inroom(dfs)
ereflex   = display_reflection_early(dfs)
hreflex   = display_reflection_horizontal(dfs)
vreflex   = display_reflection_vertical(dfs)
hspl      = display_spl_horizontal(dfs)
vspl      = display_spl_vertical(dfs)
hcontour  = display_contour_horizontal(dfs)
vcontour  = display_contour_vertical(dfs)
hradar    = display_radar_horizontal(dfs)
vradar    = display_radar_vertical(dfs)


In [ ]:
spinorama
#alt.vconcat(
#    alt.hconcat(spinorama,onaxis, inroom),
#    alt.hconcat(ereflex, hreflex, vreflex),
#    alt.hconcat(hcontour,hradar,hspl),
#    alt.hconcat(vcontour,vradar,vspl)
#)

In [ ]:
# display_spinorama(df, speaker, width, heigth)
nearest = alt.selection(
    type='single',
    nearest=True,
    on='mouseover',
    fields=['Freq'],
    empty='none')

dfu = df['Neumann KH 80']['ASR']['default']['CEA2034']

def display_graph(dfu, graph_params):
    # add selectors                                                                                                                          
    selectorsMeasurements = alt.selection_multi(
        fields=['Measurements'], 
        bind='legend')
    scales = alt.selection_interval(
        bind='scales'
    )
    
    # main charts
    xaxis = alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[graph_params['xmin'], graph_params['xmax']]))
    line=alt.Chart(dfu).mark_line(
    ).transform_filter(
        alt.FieldOneOfPredicate(field='Measurements', oneOf=['On Axis', 'Listening Window', 'Early Reflections', 'Sound Power'])
    ).encode(
        xaxis,
        alt.Y('dB:Q',   scale=alt.Scale(zero=False, domain=[graph_params['ymin'], graph_params['ymax']])),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))
    ).properties(
        width=graph_params['width'],
        height=graph_params['height']
    )
    
    di=alt.Chart(dfu).mark_line(
    ).transform_filter(
        alt.FieldOneOfPredicate(field='Measurements', oneOf=['Early Reflections DI', 'Sound Power DI'])
    ).encode(
        xaxis,
        alt.Y('dB:Q',   scale=alt.Scale(zero=False, domain=[graph_params['ymin'], graph_params['ymax']])),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))
    )
    
    circle=alt.Chart(dfu).mark_circle(
        size=100
    ).encode(
        alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[graph_params['xmin'], graph_params['xmax']])),
        alt.Y('dB:Q',   scale=alt.Scale(zero=False)),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=['Measurements', 'Freq', 'dB']
    ).transform_calculate(Freq=f'format(datum.Freq, ".0f")', dB=f'format(datum.dB, ".1f")'
    )    

    
    # assemble elements together
    #spin = (circle+di+line).add_selection(selectorsMeasurements).add_selection(scales).add_selection(nearest)
    spin = alt.layer(
        circle, line, di
    ).resolve_scale(
        y='independent'
    ).add_selection(selectorsMeasurements).add_selection(scales).add_selection(nearest)
    
    return spin

params = {'width': 900, 'height': 300, 
          'xmin': 30, 'xmax': 20000, 
          'ymin': 40, 'ymax': 110,
          'y2min': 40, 'y2max': 110,
         }
display_graph(dfu, params)


In [ ]:
# print(df.keys())
speaker1 = 'Neumann KH 80'
speaker2 = 'Kali IN-8'

def augment(dfa, name):
    namearray = [name for i in range(0,len(dfa))]
    dfa['Speaker'] = name
    return dfa

dfu = pd.concat([augment(df[k]['ASR']['default']['CEA2034'],k) for k in df.keys()])

input_dropdown1 = alt.binding_select(options=[s for s in df.keys()])
selection1 = alt.selection_single(
    fields=['Speaker'], 
    bind=input_dropdown1, 
    name='Select right ', 
    init={'Speaker': speaker1})
input_dropdown2 = alt.binding_select(options=[s for s in df.keys()])
selection2 = alt.selection_single(
    fields=['Speaker'], 
    bind=input_dropdown2, 
    name='Select left ', 
    init={'Speaker': speaker2})

line = alt.Chart(dfu).mark_line().encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[20,20000])),
    alt.Y('dB:Q',   scale=alt.Scale(zero=False)),
    alt.Color('Measurements', type='nominal', sort=None)
).properties(
    width=400,
    height=200
)

line1 = line.add_selection(selection1).transform_filter(selection1)
line2 = line.add_selection(selection2).transform_filter(selection2)

#selectors = alt.Chart(df).mark_point().encode(x='Freq:Q', opacity=alt.value(0)).add_selection(nearest)
#points = line.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))
#text = line.mark_text(align='left', dx=5, dy=-5).encode(text=alt.condition(nearest, 'dB:Q', alt.value(' ')))
#rules = alt.Chart(df).mark_rule(color='gray').encode(x='Freq:Q').transform_filter(nearest)
line2|line1 #+selectors+points+rules+text

In [ ]:
from scipy import ndimage, signal
import pandas as pd
from astropy.convolution import Gaussian2DKernel


dfu = df['Adam S2V']['SPL Horizontal_unmelted']
x,y,z = compute_contour(dfu)

def reshape2(x, y, z, nscale):
    nx, ny = x.shape
    # increase the size of the grid by nscale*nscale  
    x_min = np.min(x)
    x_max = np.max(x)
    y_min = np.min(y)
    y_max = np.max(y)
    # print('x min=',x_min, 'max=',x_max, 'y min=', y_min, 'max=',y_max, 'nscale=', nscale)
    lx = np.linspace(np.min(y), np.max(y), nx*nscale)
    lyi = [np.linspace(x[0][i],x[0][i+1],nscale, endpoint=False) for i in range(0, len(x[0])-1)]
    ly = [i for j in lyi for i in j] + [x[0][len(x[0])-1] for i in range(0,nscale)]
    rx, ry = np.meshgrid(ly, lx)
    # copy paste the values of z into rz                                                                                                                                             
    rz = np.repeat(np.repeat(z, nscale, axis=1), nscale, axis=0)
    return rx, ry, rz

nscale=2
rx, ry, rz = reshape2(x, y, z, nscale)

print('u: ', x.shape, y.shape, z.shape)
print('r: ', rx.shape, ry.shape, rz.shape)

kernel = Gaussian2DKernel(x_stddev=1, y_stddev=1) #, mode='oversample', factor=10)
#kernel = gkern(9,1)
zs = ndimage.convolve(z, kernel.array, mode='mirror')
rzs = ndimage.convolve(rz, kernel.array, mode='mirror')
unsmoothed = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'z': z.ravel()})
smoothed = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'z': zs.ravel()})
smoothed_large = pd.DataFrame({'x': rx.ravel(), 'y': ry.ravel(), 'z': rzs.ravel()})
# value are between -60 and 0
speaker_scale = [-12, -9, -8, -7, -6, -5, -4, -3, -2.5, -2, -1.5, -1, -0.5, 0]
width=800
height=360
mheight=12
msize=8
alt_x = alt.X('x:O', scale=alt.Scale(type="log"), title='Frequency 200-20k Hz')
alt_y = alt.Y('y:O', title='Angle')
alt_color = alt.Color('z:Q', scale=alt.Scale(domain=speaker_scale))
chart_unsmoothed = alt.Chart(unsmoothed).transform_filter('datum.x>200').mark_rect(size=msize, height=mheight).encode(alt_x, alt_y, alt_color).properties(width=width, height=height)
chart_smoothed = alt.Chart(smoothed).transform_filter('datum.x>200').mark_rect(size=msize, height=mheight).encode(alt_x, alt_y, alt_color).properties(width=width, height=height)
chart_smoothed_large = alt.Chart(smoothed_large).transform_filter('datum.x>200').mark_rect(size=msize*nscale, height=mheight*nscale).encode(alt_x, alt_y, alt_color).properties(width=width, height=height)
print('  z min=',np.min(z), ', max=', np.max(z))
print(' rz min=',np.min(rz), ', max=', np.max(rz))
print('rzs min=',np.min(rzs), ', max=', np.max(rzs))
alt.vconcat(chart_unsmoothed, chart_smoothed, chart_smoothed_large)
